Provide a full descriptive summary of the dataset, including information such as the number of observations, summary statistics (report values to 2 decimal places), number of variables, name and type of variables, what the variables mean, any issues you see in the data, any other potential issues related to things you cannot directly see, how the data were collected, etc. Make sure to use bullet point lists or tables to summarize the variables in an easy-to-understand format

# Data Description #

## Players.csv ##
Contains player-level information - 196 unique players (observations) and 7 variables.

| Variable Name | type | n_missing | meaning |
| :- | :- | :- | :- |
|experience |character | 0 | Player's self-reported or assinged experience level (beginner, amateur, regular, pro or veteran).|
|subscribe |logical | 0 | Whether player has subscribed to the game-related newsletter (true/false).|
|hashedEmail |character | 0 | Player's email hashed as distinct values.|
|played_hours |double | 0 | Total hours a player has played.|
|name |character | 0 |Player's display name.|
|gender |character | 0 |Self registered gender of a player.|
|Age |double | 2 |Age in years.|


#### Summarised Statistics ####

Number of different type of experienced players
| Experience | Count |
| :- | :- |
|Amateur |63|
|Beginner |35|
|Pro |14|
|Regular|36|
|Veteran|8|

Subscriber count

| Subscriber | Count |
| :- | :- |
|TRUE| 52|
|FALSE| 144|

Statistics for hours played
| count | mean| minimum | median | max |
| :- | :- | :- | :- | :- |
|196.00|5.85|0.00|0.10|223.10|

Statistics for age
| count | mean| minimum | median | max |
|:-| :- | :- | :- | :- |
|196.00|21.14|9.00|19.00|58.00|

##### Potential Issues ######
- Age: Two missing values. May cause biasness when modelling, but unlikely given the small proportion (2/196).
- played_hours: Highly skewed to the right; mean significantly exceeds the median. This may suggest there are many casual players and only a few regualr heavy players (potential outliers).
- Name: May not be useful in contributing to modelling process.
- Subscribed status: May have changed since data collection.

##### Likely Data Collection Methods #####
- Personal attributes were likely collected via survey upon account creation.
- Gameplay metrics like played_hours likely collected through aggregating server-side metrics.
- hashedEmails likely encrypted through a hashing algorithm for privacy.


## Sessions.csv ##
Contains session-level information - 1535 unique sessions (observations) and 9 variables.
| Variable Name | type | n_missing | meaning |
| :- | :- | :- | :- |
|hashedEmail |character | 0 | Player's email hashed as distinct values.|
|start_time |character | 0 | Start of session timestamp (format: day/month/year hour:minutes).|
|end_time |character | 2 |End of session timestamp.|
|original_start_time |double | 0 | Timestamp in milliseconds for start of session.|
|original_end_time |double | 2 |Timestamp in milliseconds for end of session.|


#### Summarised Statistics ####

Number of times each player (identified via number of repeated hashedEmail) played the game.
(Please see below in code appendix for convenience).

Statistics for duration of time played in minutes.
| number of sessions | average | shortest | median| max |
| :- | :- | :- | :- | :- |
|1535.00 |48.49|0.00|0.00|333.33|

##### Potential Issues ######
- Zero duration sessions: May indicate users who logged in without playing.
- Median duration = 0: suggests very short sessions which may have been rounded down or potential data recording errors.
- Only 125 out of 196 players have matching session records, indicating some users never played - may affect modeling of player-level behaviour.
- Missing end timestamps - session may have still been active when data was taken.
- Longer sessions may be due to idle time rather than continuous gameplay; not reflective of duration of gameplay.
- duplicate sessions per hashedEmail - may be due to reconnection or restarts.

##### Likely Data Collection Methods #####
- Data about session was likely logged in real time from server events marking the start and end timestamps.


# Questions #

#### Broad Question Chosen: #####
What time windows are most likely to have large number of simultaneous players? [3]

#### Specific Question ####
Can features of time (e.g. hour of day, day of week, and recent activity trends) and aggregated player characteristics (e.g. hours played and experience levels) predict the number of concurrent players active on the server during each hourly period?

##### Response Variable  #####
* concurrent_players (double) = number of unique players active during time window t.

##### Explanatory Variables #####
* Temporal features: hour of day, day of week.
* Recent activity features: number of sessions or concurrent players in previous time windows.
* Player attributes: played_hours, proportion of high-experience players.

#### How the data will be used ####
The sessions.csv dataset contains detialed session start and end times for each player, allowing for computing the number of players that overlap in the same hourly window - allowing us to achieve the count of concurrent players per hour.

The players.csv dataset provides us with the background for the behavioural attributes that can be merged using hashedEmail to gain more information about each player's profile These variabels will be aggregated to describe the composition of active players in each hourly period.

#### Data wrangling plan ####
1. Parse timestamps to be in datetime format.
2. Create hourly intervals across full observation model
3. For each interval, count unique players
4. Merge the two datasets to account for behavioural features of each player for each hour.





# APPENDIX (CODE) #

In [1]:
library(tidyverse)
library(tidymodels)
library(repr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [2]:
players <- read_csv("https://raw.githubusercontent.com/beam-9/dsci-100-2025w1-group-003-10/refs/heads/main/data/players.csv")
players

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17
Regular,TRUE,8e594b8953193b26f498db95a508b03c6fe1c24bb5251d392c18a0da9a722807,0.0,Luna,Female,19
Amateur,FALSE,1d2371d8a35c8831034b25bda8764539ab7db0f63938696917c447128a2540dd,0.0,Emerson,Male,21
Amateur,TRUE,8b71f4d66a38389b7528bb38ba6eb71157733df7d1740371852a797ae97d82d1,0.1,Natalie,Male,47


In [3]:
n_vars_players <- ncol(players)
n_obs_players <- nrow(players)

n_vars_players #num of variables
n_obs_players #num of observations

summary_exp_player <- players |>
    group_by(experience) |>
    summarize(count = n()) #num of different type of experiences for players

summary_exp_player

summary_sub_player <- players |>
    group_by(subscribe) |>
    summarize(count = n()) #num of subscribed/not subscribed players

summary_sub_player

summary_hours_players <- players |>
    summarise(count = sum(!is.na(played_hours)), #checking for na values, all counts should hypothetically = num of observations.
              mean_hrs = round(mean(played_hours, na.rm = TRUE), 2), #use round with 2 to round to 2 decimal places.
              min_hrs = round(min(played_hours, na.rm = TRUE), 2),
              median_hrs = round(median(played_hours, na.rm = TRUE), 2),
              max_hrs = round(max(played_hours, na.rm = TRUE), 2))

summary_hours_players

summary_gender_players <- players |>
    group_by(gender) |>
    summarise(count = n()) #num of different genders

summary_gender_players

summary_age_players <- players |>
  summarise(count = sum(!is.na(Age)),
              mean_age = round(mean(Age, na.rm = TRUE), 2),
              min_age = round(min(Age, na.rm = TRUE), 2),
              median_age = round(median(Age, na.rm = TRUE), 2),
              max_age = round(max(Age, na.rm = TRUE), 2))

summary_age_players

tidy_players <- players |> #players.csv without the two observations with N/A values for Age.
    filter(!is.na(Age))

[1] 7

[1] 196

experience,count
<chr>,<int>
Amateur,63
Beginner,35
Pro,14
Regular,36
Veteran,48


subscribe,count
<lgl>,<int>
FALSE,52
TRUE,144


count,mean_hrs,min_hrs,median_hrs,max_hrs
<int>,<dbl>,<dbl>,<dbl>,<dbl>
196,5.85,0,0.1,223.1


gender,count
<chr>,<int>
Agender,2
Female,37
Male,124
Non-binary,15
Other,1
Prefer not to say,11
Two-Spirited,6


count,mean_age,min_age,median_age,max_age
<int>,<dbl>,<dbl>,<dbl>,<dbl>
194,21.14,9,19,58


In [4]:
sessions <- read_csv("https://raw.githubusercontent.com/beam-9/dsci-100-2025w1-group-003-10/refs/heads/main/data/sessions.csv")
head(sessions)

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


In [5]:
n_obs_sessions <- nrow(sessions)
n_vars_sessions <- ncol(sessions)

n_obs_sessions #num of observations
n_vars_sessions #num of variables

summary_email_sessions <- sessions |>
    group_by(hashedEmail) |>
    summarize(count = n()) |> #how often a player re entered and played the game 
    arrange(-count)

summary_email_sessions 

sessions <- sessions |>
  mutate(
      duration_ms = original_end_time - original_start_time,
      duration_min = round(duration_ms / 1000 / 60, 2))

# Compute summary statistics for session durations (in minutes)
session_duration_summary <- sessions |>
  summarise(
    count_sessions = n(),
    average_duration_min = round(mean(duration_min, na.rm = TRUE), 2),
    shortest_duration_min = round(min(duration_min, na.rm = TRUE), 2),
    longest_duration_min = round(max(duration_min, na.rm = TRUE), 2),
    median_duration_min = round(median(duration_min, na.rm = TRUE), 2))

session_duration_summary

[1] 1535

[1] 5

hashedEmail,count
<chr>,<int>
fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33cbb5e894a3867ca44d,310
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,219
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,159
ad6390295640af1ed0e45ffc58a53b2d9074b0eea694b16210addd44d7c81f83,147
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,130
b622593d2ef8b337dc554acb307d04a88114f2bf453b18fb5d2c80052aeb2319,95
24d4892c2c3ce11d3e54c3bf31ee218901cac7a2f564911d67424c8fdf0874fe,79
e74c60a92c0100e7240be56d66969db85856152b048c639331a3595f901461ae,47
18936844e06b6c7871dce06384e2d142dd86756941641ef39cf40a9967ea14e3,41


count_sessions,average_duration_min,shortest_duration_min,longest_duration_min,median_duration_min
<int>,<dbl>,<dbl>,<dbl>,<dbl>
1535,48.49,0,333.33,0
